In [532]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [533]:
base = pd.read_csv('iec6100429.csv')
base.head()

,Amostra 1,Amostra 2,Amostra 3,Amostra 4,Amostra 5,Amostra 6,Amostra 7,Amostra 8,Amostra 9,Amostra 10,...,Amostra 183,Amostra 184,Amostra 185,Amostra 186,Amostra 187,Amostra 188,Amostra 189,Amostra 190,Amostra 191,Forma de onda
0,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,1.0
1,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,1.0
2,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,1.0
3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,1.0
4,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,1.0


In [534]:
x = base.drop('Forma de onda', axis=1) # atributos previsores
y = base.iloc[:,-1] # pegando a ultima coluna para saber qual a classe

In [535]:
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
x = scaler_x.fit_transform(x)
x.shape

(15, 191)

In [536]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(categories = 'auto')
y.shape

(15,)

In [537]:
y = y.values.reshape(-1,1)
y.shape

(15, 1)

In [538]:
y = onehot.fit_transform(y).toarray()

In [539]:
from sklearn.model_selection import train_test_split
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, test_size = 0.5)

In [540]:
x.shape, x_teste.shape, y.shape, y_teste.shape

((15, 191), (8, 191), (15, 3), (8, 3))

In [541]:
neuronios_entrada = x.shape[1]
neuronios_entrada

191

In [542]:
neuronios_oculta = 30
neuronios_oculta

30

In [543]:
neuronios_saida = y.shape[1]
neuronios_saida

3

In [544]:
w = {'oculta': tf.Variable(tf.random_normal([neuronios_entrada, neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_oculta, neuronios_saida]))}

In [545]:
b = {'oculta': tf.Variable(tf.random_normal([neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_saida]))}

In [546]:
xph = tf.placeholder('float', [None, neuronios_entrada])
yph = tf.placeholder('float', [None, neuronios_saida])

In [547]:
def mlp(x, w, bias):
    camada_oculta = tf.add(tf.matmul(x, w['oculta']), b['oculta'])
    camada_oculta_ativacao = tf.nn.relu(camada_oculta)
    camada_saida = tf.add(tf.matmul(camada_oculta_ativacao, w['saida']), b['saida'])
    return camada_saida

In [548]:
modelo = mlp(xph, w, b)

In [549]:
erro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = modelo, labels = yph))
otimizador = tf.train.AdamOptimizer(learning_rate = 0.1).minimize(erro)

In [550]:
batch_size = 5
batch_total = int(len(x_treinamento) / batch_size)
batch_total

1

In [551]:
x_batches = np.array_split(x_treinamento, batch_total)

In [552]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoca in range(5000):
        erro_medio = 0.0
        batch_total = int(len(x_treinamento) / batch_size)
        x_batches = np.array_split(x_treinamento, batch_total)
        y_batches = np.array_split(y_treinamento, batch_total)
        for i in range(batch_total):
            x_batch, y_batch = x_batches[i], y_batches[i]
            _, custo = sess.run([otimizador, erro], feed_dict = {xph: x_batch, yph: y_batch})
            erro_medio += custo / batch_total
        if epoca % 500 == 0:
            print('Época: ' + str(epoca+1) + ' erro: '+ str(erro_medio))
    w_final, b_final = sess.run([w,b])

Época: 1 erro: 31.675386428833008
Época: 501 erro: 0.00032789265969768167
Época: 1001 erro: 9.433005470782518e-05
Época: 1501 erro: 4.333780816523358e-05
Época: 2001 erro: 2.675274845387321e-05
Época: 2501 erro: 1.7608384951017797e-05
Época: 3001 erro: 1.200583119498333e-05
Época: 3501 erro: 8.41264863993274e-06
Época: 4001 erro: 6.011491677782033e-06
Época: 4501 erro: 4.359621470939601e-06


In [553]:
previsoes_teste = mlp(xph, w_final, b_final)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    r1 = sess.run(previsoes_teste, feed_dict = {xph: x_teste})
    r2 = sess.run(tf.nn.softmax(r1))
    r3 = sess.run(tf.argmax(r2, 1))

In [554]:
y_teste2 = np.argmax(y_teste, 1)

In [555]:
from sklearn.metrics import accuracy_score
taxa_acerto = accuracy_score(y_teste2, r3)
taxa_acerto

0.125